In [1]:
!pip install torchtext==0.12.0
!pip install torchdata
!pip install pytorch-lightning
!pip install wandb



import torchtext
import spacy
import torch
import torch.nn as nn
import torchtext.transforms as T
import torch.optim as optim
import pytorch_lightning as pl
import torch.nn.functional as F

import warnings
# warnings.filterwarnings("ignore")

print(torchtext.__version__)

0.12.0


In [2]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


class RNN(pl.LightningModule):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim,
                 train_datapipe, val_datapipe, test_datapipe, batch_size=32,n_hidden = 1):
        super().__init__()
        
        # Required since our input vector represents each word as an index into
        # the vocabulary.
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=1)
        # Creates an RNN using tanh by default.
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers= n_hidden, batch_first=True)

        self.fc = nn.Linear(hidden_dim, output_dim)

        # LightningModule attributes
        self.lr = 1e-3
        self.batch_size = batch_size
        self.loss_fn = nn.BCEWithLogitsLoss()

        # Datasets
        self.train_datapipe = train_datapipe
        self.val_datapipe = val_datapipe
        self.test_datapipe = test_datapipe

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.rnn(embedded)

        return self.fc(hidden.squeeze(0))

    def training_step(self, batch, batch_idx):
        input = batch["token_ids"].cuda()
        target = torch.tensor(batch["target"], dtype=torch.float).cuda()
        output = self(input).squeeze()
        loss = self.loss_fn(output, target)
        acc = binary_accuracy(output, target)
        
        self.log("train_loss", loss)
        self.log("train_acc", acc)

        return loss
    
    def validation_step(self, batch, batch_idx):
        input = batch["token_ids"].cuda()
        target = torch.tensor(batch["target"], dtype=torch.float).cuda()
        output = self(input).squeeze()
        loss = self.loss_fn(output, target)
        acc = binary_accuracy(output, target)
        
        self.log("val_loss", loss)
        self.log("val_acc", acc)

    def train_dataloader(self):
        loader = torch.utils.data.DataLoader(self.train_datapipe,
                                             batch_size=None,
                                             num_workers=8,
                                             shuffle=True)

        return loader

    def val_dataloader(self):
        loader = torch.utils.data.DataLoader(self.val_datapipe,
                                             batch_size=None,
                                             num_workers=8,
                                             shuffle=False)

        return loader

    def test_dataloader(self):
        loader = torch.utils.data.DataLoader(self.test_datapipe,
                                             batch_size=None,
                                             num_workers=8,
                                             shuffle=False)

        return loader

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

In [3]:
tokenizer = torchtext.data.utils.get_tokenizer("spacy", language="en_core_web_sm")
max_tokens = 10000

def make_vocabulary():
    train_dataset = torchtext.datasets.SST2(split="train")
    train_datapipe = train_dataset.map(lambda x: tokenizer(x[0]))
    v = torchtext.vocab.build_vocab_from_iterator(train_datapipe, specials=["<unk>"], max_tokens=max_tokens)
    v.set_default_index(0)

    return v

In [4]:
v = make_vocabulary()


In [5]:
padding_idx = 1
bos_idx = 0
eos_idx = 2
max_seq_len = 256

text_transform = T.Sequential(
    T.VocabTransform(v),
    T.Truncate(max_seq_len - 2),
    T.AddToken(token=bos_idx, begin=True),
    T.AddToken(token=eos_idx, begin=False),
    T.ToTensor(padding_value=padding_idx)
)

batch_size = 32

train_datapipe = torchtext.datasets.SST2(split='train')
val_datapipe = torchtext.datasets.SST2(split='dev')
test_datapipe = torchtext.datasets.SST2(split='test')

train_datapipe = train_datapipe.map(lambda x: (tokenizer(x[0]), x[1]))
train_datapipe = train_datapipe.batch(batch_size).rows2columnar(["text", "label"])
train_datapipe = train_datapipe.map(lambda x: {"token_ids": text_transform(x["text"]), "target": x["label"]})

val_datapipe = val_datapipe.map(lambda x: (tokenizer(x[0]), x[1]))
val_datapipe = val_datapipe.batch(batch_size).rows2columnar(["text", "label"])
val_datapipe = val_datapipe.map(lambda x: {"token_ids": text_transform(x["text"]), "target": x["label"]})

test_datapipe = test_datapipe.map(lambda x: (tokenizer(x[0]), x[1]))
test_datapipe = test_datapipe.batch(batch_size).rows2columnar(["text", "label"])
test_datapipe = test_datapipe.map(lambda x: {"token_ids": text_transform(x["text"]), "target": x["label"]})

In [6]:
model = RNN(len(v), 100, 256, 1, train_datapipe, val_datapipe, test_datapipe,n_hidden = 2)


In [7]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import wandb
key  = wandb.login()
print(key)
wandb_logger1 = WandbLogger(project="RNN-model2")


checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
)


trainer = pl.Trainer(accelerator="gpu", callbacks=[checkpoint_callback], max_epochs=20,logger=wandb_logger1 )
trainer.fit(model)


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True


...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | embedding | Embedding         | 1.0 M 
1 | rnn       | RNN               | 223 K 
2 | fc        | Linear            | 257   
3 | loss_fn   | BCEWithLogitsLoss | 0     
------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.894     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [8]:
model = RNN(len(v), 100, 256, 1, train_datapipe, val_datapipe, test_datapipe,n_hidden = 1)


In [9]:

key  = wandb.login()
print(key)
wandb_logger_2 = WandbLogger(project="RNN-model-1-f")


checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
)


trainer = pl.Trainer(accelerator="gpu", callbacks=[checkpoint_callback], max_epochs=2,logger=wandb_logger_2 )
trainer.fit(model)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/wandb.py:348: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | embedding | Embedding         | 1.0 M 
1 | rnn       | RNN               | 91.6 K
2 | fc        | Linear            | 257   
3 | loss_fn   | BCEWithLogitsLoss | 0     
------------------------------------------------
1.1 M     Trainable params
0         Non-train

True


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


ValueError: ignored

# Creating a LSTM Class

In [13]:
# def binary_accuracy(preds, y):
#     """
#     Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
#     """

#     #round predictions to the closest integer
#     rounded_preds = torch.round(torch.sigmoid(preds))
#     correct = (rounded_preds == y).float() #convert into float for division 
#     acc = correct.sum() / len(correct)
#     return acc


class LSTM(pl.LightningModule):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim,
                 train_datapipe, val_datapipe, test_datapipe, batch_size=32):
        super().__init__()
        
        # Required since our input vector represents each word as an index into
        # the vocabulary.
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=1)
        # Creates an RNN using tanh by default.
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

        # LightningModule attributes
        self.lr = 1e-3
        self.batch_size = batch_size
        self.loss_fn = nn.BCEWithLogitsLoss()

        # Datasets
        self.train_datapipe = train_datapipe
        self.val_datapipe = val_datapipe
        self.test_datapipe = test_datapipe

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.rnn(embedded)

        return self.fc(hidden.squeeze(0))

    def training_step(self, batch, batch_idx):
        input = batch["token_ids"].cuda()
        target = torch.tensor(batch["target"], dtype=torch.float).cuda()
        output = self(input).squeeze()
        loss = self.loss_fn(output, target)
        acc = binary_accuracy(output, target)
        
        self.log("train_loss", loss)
        self.log("train_acc", acc)

        return loss
    
    def validation_step(self, batch, batch_idx):
        input = batch["token_ids"].cuda()
        target = torch.tensor(batch["target"], dtype=torch.float).cuda()
        output = self(input).squeeze()
        loss = self.loss_fn(output, target)
        acc = binary_accuracy(output, target)
        
        self.log("val_loss", loss)
        self.log("val_acc", acc)

    def train_dataloader(self):
        loader = torch.utils.data.DataLoader(self.train_datapipe,
                                             batch_size=None,
                                             num_workers=8,
                                             shuffle=True)

        return loader

    def val_dataloader(self):
        loader = torch.utils.data.DataLoader(self.val_datapipe,
                                             batch_size=None,
                                             num_workers=8,
                                             shuffle=False)

        return loader

    def test_dataloader(self):
        loader = torch.utils.data.DataLoader(self.test_datapipe,
                                             batch_size=None,
                                             num_workers=8,
                                             shuffle=False)

        return loader

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

In [14]:
model_ls = LSTM(len(v), 100, 256, 1, train_datapipe, val_datapipe, test_datapipe)

In [12]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import wandb
key  = wandb.login()
print(key)
wandb_logger_3 = WandbLogger(project="RNN-lstm-1")


checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
)


trainer = pl.Trainer(accelerator="gpu", callbacks=[checkpoint_callback], max_epochs=20,logger=wandb_logger_3 )
trainer.fit(model_ls)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/wandb.py:348: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /content/RNN-lstm-1/2iac9ikj/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
-----------------------------------------------

True


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd41b92f8c0>
Traceback (most recent call last):
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd41b92f8c0>
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd41b92f8c0>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in _

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]